# Influential Cases Template

Recipe `interpretability_overview.ipynb` provides a basic introduction to the Howso Engine interpretability capabilities. This recipe provides a barebones template to retrieve Influential Cases.


In [1]:
import pandas as pd
from pmlb import fetch_data

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

# Section 1: Train, Analyze, and React

We will be using the `Adult` dataset where the Action Feature is a binary indicator of whether a person makes over $50k/year.

### Step 1: Load Data

In [2]:
df = fetch_data('adult', local_cache_dir="../../../data/adult")

# Subsample the data to ensure the example runs quickly
df = df.sample(1000, random_state=0).reset_index(drop=True)

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,41.0,4,151856.0,11,9.0,2,11,0,4,1,0.0,0.0,40.0,39,1
1,57.0,6,87584.0,10,16.0,0,10,1,4,0,0.0,0.0,25.0,39,1
2,31.0,2,220669.0,9,13.0,4,10,3,4,0,6849.0,0.0,40.0,39,1
3,55.0,4,171355.0,8,11.0,2,7,0,4,1,0.0,0.0,20.0,39,1
4,59.0,6,148626.0,0,6.0,2,5,0,4,1,0.0,0.0,40.0,39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,29.0,4,166220.0,9,13.0,4,12,3,4,0,0.0,0.0,50.0,39,1
996,60.0,0,204486.0,9,13.0,2,0,0,4,1,0.0,0.0,8.0,39,0
997,48.0,2,93449.0,14,15.0,2,10,0,1,1,99999.0,0.0,40.0,30,0
998,25.0,4,114838.0,14,15.0,4,10,1,4,0,0.0,0.0,8.0,22,1


### Step 1a: Define Test Case

In [3]:
# Split out the last row for a test set
df_test = df.iloc[[-1]].copy()
df_test = df_test.drop('target', axis=1)

df.drop(df.index[-1], inplace=True)

### Step 2: Train

In [4]:
# Infer feature attributes
features = infer_feature_attributes(df)

# Specify Context and Action Features
action_features = ['target']
context_features = features.get_names(without=action_features)

# Create the Trainee
t = Trainee(
    features=features,
    overwrite_existing=True
)

# Train
t.train(df)

# Targeted Analysis
t.analyze(context_features=context_features, action_features=action_features)

# Influential Cases

To determine influential cases, use `React` on specific cases. For ease of understanding, we use one case here.

In [5]:
# Specify the set of details to request from the Trainee that will
# provide interpretability
details = {
    'influential_cases': True,
}

results = t.react(
    df_test,
    context_features=context_features,
    action_features=action_features,
    details=details
)

In [6]:
influential_cases = pd.DataFrame(results['details']['influential_cases'][0])
